Table 4.11.1: Housing situation of new presentations to treatment

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.11.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<F5 14514.0>, <H11 50656.0>, <F9 18592.0>, <B9 40782.0>, <G6 0.12322504302926>, <J6 14037.0>, <F7 1013.0>, <F8 774.0>, <I8 0.0043662898498714>, <I9 1.0>, <H9 50157.0>, <J10 1287.0>, <H6 3681.0>, <J9 126020.0>, <G7 0.0544858003442341>, <E8 0.0504578810115835>, <E9 1.0>, <K6 0.111387081415648>, <B5 27931.0>, <J7 9767.0>, <I6 0.0733895567916741>, <F11 18757.0>, <H7 1524.0>, <K7 0.077503570861768>, <C8 0.00284439213378451>, <H8 219.0>, <E5 0.794287100491237>, <F10 165.0>, <D5 13097.0>, <E7 0.0465158590575535>, <B11 41178.0>, <I5 0.891859560978527>, <K9 1.0>, <G8 0.0416308089500861>, <H10 499.0>, <C6 0.153793340199107>, <J8 1941.0>, <C5 0.684885488695993>, <F6 2291.0>, <D8 832.0>, <D6 1793.0>, <B10 396.0>, <J5 100275.0>, <C7 0.158476778971115>, <B7 6463.0>, <K8 0.015402317092525>, <D9 16489.0>, <G9 1.0>, <G5 0.78065834767642>, <D7 767.0>, <I7 0.030384592379927>, <E6 0.108739159439626>, <B8 116.0>, <D11 16716.0>, <D10 227.0>, <C9 1.0>, <B6 6272.0>, <J11 127307.0>, <K5 0.795707030630059>, <H

In [4]:
house = tab.excel_ref('A5').expand(DOWN).is_not_blank() - tab.excel_ref('A11').expand(DOWN)
house

{<A9 'Total'>, <A8 'Other'>, <A10 'Inconsistent/missing'>, <A6 'Housing problem'>, <A7 'Urgent housing problem (NFA)'>, <A5 'No problem'>}

In [5]:
Clients = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
Clients

{<D3 'Non-opiate only'>, <H3 'Alcohol only'>, <J3 'Total'>, <B3 'Opiate'>, <F3 'Non-opiate and Alcohol'>}

In [6]:
MeasureType = tab.excel_ref('B4').expand(RIGHT).is_not_blank()
MeasureType

{<B4 'n'>, <K4 '%'>, <E4 '%'>, <J4 'n'>, <D4 'n'>, <G4 '%'>, <F4 'n'>, <I4 '%'>, <C4 '%'>, <H4 'n'>}

In [7]:
Dimensions = [
            HDim(house,'Housing Status',DIRECTLY,LEFT),
            HDim(Clients,'Clients in new treatment',CLOSEST,LEFT),
            HDim(MeasureType,'Measure Type',DIRECTLY,ABOVE),
            HDimConst('Unit','People')            
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()
new_table

NoLookupError: No header found for <B11 41178.0>

In [ ]:
new_table = new_table[new_table['OBS'] != 0 ]

In [ ]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [ ]:
new_table.head()

In [ ]:
new_table['Measure Type'] = new_table['Measure Type'].map(
    lambda x: {
        'n' : 'Count', 
        '%' : 'Percentage',
        }.get(x, x))

In [ ]:
new_table.tail()

In [ ]:
new_table.dtypes

In [ ]:
new_table['Value'] = new_table['Value'].astype(str)

In [ ]:
new_table.head(3)

In [ ]:
new_table['Housing Status'] = new_table['Housing Status'].map(
    lambda x: {
        'Total' : 'All' 
        }.get(x, x))

In [ ]:
def user_perc(x):
    
    if x == None:
        return 'All inclusice Inconsistent/missing'
    else:
        return x
    
new_table['Housing Status'] = new_table.apply(lambda row: user_perc(row['Housing Status']), axis = 1)

In [ ]:
new_table['Clients in new treatment'] = new_table['Clients in new treatment'].map(
    lambda x: {
        'Total' : 'All Clients',
        }.get(x, x))

In [ ]:
new_table.head(3)

In [ ]:
new_table['Housing Status'] =  'Housing situation/' + new_table['Housing Status']
new_table['Clients in new treatment'] = 'New clients/' + new_table['Clients in new treatment']

In [ ]:
new_table.columns = ['Clients in treatment' if x=='Clients in new treatment' else x for x in new_table.columns]
new_table['Period'] = '2017-18'
new_table.columns = ['Basis of treatment' if x=='Housing Status' else x for x in new_table.columns]
new_table['Substance'] = 'All'
new_table = new_table[['Period','Basis of treatment','Substance','Clients in treatment','Measure Type','Value','Unit']]

In [ ]:
new_table.head()